In [7]:
# Tune learning_rate
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
# load data
dataset = loadtxt('pima-indians-diabetes.csv', delimiter=",")
# split data into X and y
X = dataset[:,0:8]
Y = dataset[:,8]
# grid search
# model = XGBClassifier()
model = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)

positives = sum(Y)
negs = Y.shape[0]


In [12]:
full_param_grid =  {
            'min_child_weight': [1, 5, 10],
            'gamma': [0.5, 1, 1.5, 2, 5],
            'subsample': [0.6, 0.8, 1.0],
            'colsample_bytree': [0.6, 0.8, 1.0],
            'max_depth': [3, 4, 5],
            'n_estimators':[10,500,800,1000],
            'scale_pos_weight' : [1, negs/positives, positives/negs]
        }


def random_grid_searching_model(model,  X, Y, folds=5, params=full_param_grid, param_comb=5):
    from sklearn.model_selection import RandomizedSearchCV
    skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)
    
    random_search = RandomizedSearchCV(
        model, 
        param_distributions=params, 
        n_iter=param_comb, 
        scoring='roc_auc', 
        n_jobs=-1, 
        cv=skf.split(X,Y), 
        verbose=3, 
        random_state=1001 
    )

    # Here we go
    start_time = _timer(None)
    random_search.fit(X, Y)
    print (_timer(start_time)) 

    print('\n Best estimator:')
    print(random_search.best_estimator_)
    print('\n Best score:')
    print(random_search.best_score_ )
    print('\n Best hyperparameters:')
    print(random_search.best_params_)

    return random_search


def _timer(start_time=None):
    from datetime import datetime
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))



narrowed_param_grid =  {
            'min_child_weight': [5, 10],
            'gamma': [1, 1.5, 2],
            'subsample': [0.8],
            'colsample_bytree': [0.8, 0.6],
            'max_depth': [3, 5],
            'n_estimators':[500,800,1000],
            'scale_pos_weight' : [1, negs/positives, positives/negs]
        }

In [13]:
random_grid_searching_model(model, X, Y,params=full_param_grid)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] subsample=0.8, scale_pos_weight=1, n_estimators=10, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=0.8 
[CV] subsample=0.8, scale_pos_weight=1, n_estimators=10, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=0.8 
[CV] subsample=0.8, scale_pos_weight=1, n_estimators=10, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=0.8 
[CV] subsample=0.8, scale_pos_weight=1, n_estimators=10, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=0.8 
[CV]  subsample=0.8, scale_pos_weight=1, n_estimators=10, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=0.8, score=0.8393518518518519, total=   0.0s
[CV]  subsample=0.8, scale_pos_weight=1, n_estimators=10, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=0.8, score=0.7707407407407407, total=   0.0s
[CV]  subsample=0.8, scale_pos_weight=1, n_estimators=10, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=0.8, score=0.78

[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    2.8s finished



 Time taken: 0 hours 0 minutes and 3.13 seconds.
None

 Best estimator:
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=1, learning_rate=0.02, max_delta_step=0,
       max_depth=3, min_child_weight=5, missing=None, n_estimators=500,
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=0.3489583333333333,
       seed=None, silent=True, subsample=0.8)

 Best score:
0.8279980400506638

 Best hyperparameters:
{'subsample': 0.8, 'scale_pos_weight': 0.3489583333333333, 'n_estimators': 500, 'min_child_weight': 5, 'max_depth': 3, 'gamma': 1, 'colsample_bytree': 0.8}


RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x1a167c7f68>,
          error_score='raise',
          estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.02, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=600,
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
          fit_params=None, iid=True, n_iter=5, n_jobs=-1,
          param_distributions={'min_child_weight': [1, 5, 10], 'gamma': [0.5, 1, 1.5, 2, 5], 'subsample': [0.6, 0.8, 1.0], 'colsample_bytree': [0.6, 0.8, 1.0], 'max_depth': [3, 4, 5], 'n_estimators': [10, 500, 800, 1000], 'scale_pos_weight': [1, 2.8656716417910446, 0.3489583333333333]},
          pre_dispatch='2*n_jobs', random_state=1001, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=3)

In [20]:

narrowed_param_grid =  {
        'min_child_weight': [5],
        'gamma': [1,2],
        'subsample': [0.8],
        'colsample_bytree': [0.8],
        'max_depth': [3],
        'n_estimators':[500,],
        'scale_pos_weight' : [1, negs/positives, positives/negs]
        }

def full_gridsearch(model,  X, Y,  param_grid, folds=5,):
    
    from sklearn.model_selection import GridSearchCV
    skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)
    grid_search = GridSearchCV(model, param_grid, scoring="roc_auc", n_jobs=-1, cv=skf.split(X,Y))
    grid_result = grid_search.fit(X, Y)
    
    
    
    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']

    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

    return grid_result

full_gridsearch(model, X, Y,param_grid=narrowed_param_grid)

Best: 0.831581 using {'colsample_bytree': 0.8, 'gamma': 2, 'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 500, 'scale_pos_weight': 0.3489583333333333, 'subsample': 0.8}
0.821376 (0.023360) with: {'colsample_bytree': 0.8, 'gamma': 1, 'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 500, 'scale_pos_weight': 1, 'subsample': 0.8}
0.821140 (0.023506) with: {'colsample_bytree': 0.8, 'gamma': 1, 'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 500, 'scale_pos_weight': 2.8656716417910446, 'subsample': 0.8}
0.827998 (0.025943) with: {'colsample_bytree': 0.8, 'gamma': 1, 'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 500, 'scale_pos_weight': 0.3489583333333333, 'subsample': 0.8}
0.825108 (0.023456) with: {'colsample_bytree': 0.8, 'gamma': 2, 'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 500, 'scale_pos_weight': 1, 'subsample': 0.8}
0.821213 (0.022976) with: {'colsample_bytree': 0.8, 'gamma': 2, 'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 500, 'scal

GridSearchCV(cv=<generator object _BaseKFold.split at 0x1a16886f68>,
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.02, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=600,
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'min_child_weight': [5], 'gamma': [1, 2], 'subsample': [0.8], 'colsample_bytree': [0.8], 'max_depth': [3], 'n_estimators': [500], 'scale_pos_weight': [1, 2.8656716417910446, 0.3489583333333333]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)